In [1]:
import os
import sys
import pandas as pd
import numpy as np
import regex as re
import mygene
from matplotlib import pyplot as plt

In [2]:
mg = mygene.MyGeneInfo()

In [3]:
#mg.getgene("ENSG00000198938", 'name,symbol,refseq.rna,type_of_gene,exac.bp')

In [4]:
working_dir = "/Users/filippo/Developer/tesi"
os.chdir(working_dir)
dirs = os.listdir("data")

In [5]:
len(dirs)

8935

In [6]:
df = pd.read_csv(("%s/genes.txt"%working_dir), header=[0], index_col=[0])
df.columns=["type_of_gene"]
print(len(df))
df.head()

60483


,type_of_gene
ENSG00000242268,intergenic
ENSG00000270112,unknown
ENSG00000167578,protein-coding
ENSG00000273842,unknown
ENSG00000078237,protein-coding


In [7]:
uniq = [g[:15] for g in df.index.values]

In [8]:
len(np.unique(uniq))

60483

In [11]:
df = pd.read_csv(("%s/mainTable.csv"%working_dir))
genes = np.array([gene[:15] for gene in df.loc[:,'gene'].values])
df['gene'] = genes
#df = df.to_sparse(fill_value=0.)

In [12]:
df.head()

,gene,85593c7d-7441-4d41-81d3-aaa17c7e398b.F,bfeaa9f7-e413-40b8-86ba-c9d26a96c240.F,426b94fe-fcff-49f2-9d0b-c7cb49bb5559.F,38083e21-7761-47c7-9112-769c7f85e283.F,6e2570b0-fc95-4ba9-a8a5-b79f6dea0246.F,21800024-cf76-4185-b57b-526539ccdba2.F,1f0f7598-d8c0-41ba-8b07-c958fecbb5f7.F,ea50d2da-4f84-46dc-8036-c34392793aed.F,7b2c7499-15c0-4926-a12c-4561df34954b.F,...,bd556315-a050-465f-9a41-bdebe7e3eb61.F,fde7c419-b8c0-4e04-8512-0e3b8eadebda.F,46551ef6-d1de-43c3-8ea5-5ade8a5bffd7.F,1a587b92-0a11-47f0-83c2-77a2b6de1088.F,171f664b-3226-4944-9219-91d2a7fdfb5f.F,9c77be54-d58e-4dec-8894-acb71a12cac5.F,2556ce90-db5f-4beb-897a-db3cf556e054.F,234724ab-6f82-4ac9-9948-37ffd9c3e418.F,cd550979-69b5-43d0-9724-cc25eb502cb8.F,03872f69-93f5-4d6c-aeb3-dadbc2762682.F
0,ENSG00000277889,0.0,0.000000,0.000000,0.000000,3.255308,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
1,ENSG00000263834,0.0,0.765871,10.416107,0.420383,23.487668,0.000000,1.121337,0.000000,3.671882,...,0.852685,0.620916,0.250093,4.610736,0.000000,0.205483,1.598151,2.165402,2.29826,0.000000
2,ENSG00000263631,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
3,ENSG00000277073,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
4,ENSG00000266392,0.0,0.320126,0.000000,2.372161,0.000000,1.053509,2.460711,2.439732,0.727014,...,2.138481,0.259536,0.940828,1.284826,0.563846,4.122701,0.000000,3.394182,2.59375,1.242914


In [13]:
ngenes = len(df['gene'])
nfiles = len(df.loc[0,:])-1
print("genes:%d\trealizations:%d"%(ngenes,nfiles))

genes:1424	realizations:5000


## Means sigmas
thresholds are applied for FPKM between 0.1 and 1e5

In [ ]:
%timeit
means = []
for i,g in enumerate(df.loc[:,df.keys()[0]].values):
    means.append(np.average(df.loc[i,df.keys()[1:]].values))
    if(i%100==0):
        print("elapsed %d"%i)

In [ ]:
%timeit
variances = []
for i,g in enumerate(df.loc[:,df.keys()[0]].values):
    variances.append(np.var(df.loc[i,df.keys()[1:]].values))
    if(i%100==0):
        print("elapsed %d"%i)

In [ ]:
df_mv = pd.DataFrame(data=np.array(means), columns=['mean'], index=df.loc[:,df.keys()[0]].values)
df_mv.insert(1, 'variance', np.array(variances))
df_mv.head()
df_mv.to_csv("meanVariances.csv")

## manipulate meansigmas.csv

In [40]:
df_mv = pd.read_csv("meanVariances.csv", index_col = [0])
df_mv.head()

,mean,type_of_gene,variance
ENSG00000242268,0.165158,intergenic,0.292284
ENSG00000270112,0.028165,unknown,0.016174
ENSG00000167578,3.861123,protein-coding,6.342788
ENSG00000273842,0.006980,unknown,0.006047
ENSG00000078237,4.457380,protein-coding,17.953745


### Add type of gene

In [41]:
gdf = pd.read_csv("genes.txt", index_col=[0])
gdf.head()

,type_of_gene
ENSG00000242268,intergenic
ENSG00000270112,unknown
ENSG00000167578,protein-coding
ENSG00000273842,unknown
ENSG00000078237,protein-coding


In [42]:
len(gdf)

60483

In [33]:
df_mv = df_mv.drop(labels='type_of_gene', axis=1)
df_mv.insert(1,'type_of_gene', '')
for g in df_mv[df_mv['type_of_gene']==type_of_gene].index.values:
    df_mv.at[g, 'type_of_gene'] = gdf.loc[g[:15]]['type_of_gene']

In [43]:
df_mv = df_mv[df_mv['type_of_gene']==type_of_gene]
df_mv.to_csv("meanVariances.csv", index=True, header=True)

### Add occurence

In [44]:
odf = pd.read_csv("O.dat", header=None)
#print(len(df_mv[df_mv['type_of_gene']=='protein-coding']))
odf.head()

,0
0,0.0406
1,0.7424
2,0.0094
3,0.0460
4,0.8452


In [45]:
len(np.array(odf.loc[:,0].values*5000))

1424

In [46]:
type_of_gene = 'microRNA'

In [47]:
len(gdf[gdf['type_of_gene']==type_of_gene].index)

1424

In [48]:
len(df_mv.index)

1424

In [49]:
gdfpc = gdf.loc[gdf['type_of_gene']==type_of_gene]
gdfpc.insert(1, 'occurrence', np.array(odf.loc[:,0].values*5000))
#df_mv.insert(3,'occurrence', 0)
for g in df_mv.index.values:
    df_mv.at[g, 'occurrence'] = gdfpc.loc[g[:15]]['occurrence']

In [50]:
df_mv.head()

,mean,type_of_gene,variance,occurrence
ENSG00000277889,0.012846,microRNA,0.008601,203.0
ENSG00000263834,1.192730,microRNA,4.501360,3712.0
ENSG00000263631,0.004190,microRNA,0.002808,47.0
ENSG00000277073,0.013996,microRNA,0.007812,230.0
ENSG00000266392,1.741025,microRNA,7.022659,4226.0


In [51]:
df_mv.to_csv("meanVariances.csv", header=True, index=True)

## clean floating points

In [52]:
df.to_csv("mainTable.csv", index=False, header=True, float_format="%.1f")